# Base layer reprojection for km105 (PSAD)

Clean workflow to clip/reproject catchment, DEM, rivers, and geology for the km105 catchment. All outputs land under `mountain/reprojected` in PSAD (EPSG:24893). Run cells top to bottom; NetCDF steps stay in a separate notebook as requested.

In [1]:
from pathlib import Path

import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio
from rasterio import features
import rioxarray as rxr
from shapely.geometry import box

PSAD_CRS = "EPSG:24893"
BUFFER_METERS = 2000  # 2 km each side for the bbox buffer

# Paths
mountain_dir = Path('/home/jec18/modflow/jupy/VUB/mountain')
out_dir = mountain_dir / 'reprojected'
vectors_dir = out_dir / 'vectors'
rasters_dir = out_dir / 'rasters'
for p in (out_dir, vectors_dir, rasters_dir):
    p.mkdir(parents=True, exist_ok=True)

catchment_path = Path('/home/jec18/catchments/km105.shp')
dem_path = Path('/home/jec18/HydroSheds/sa_dem_3s.tif')
rivers_path = Path('/home/jec18/HydroSheds/HydroRIVERS_v10_sa_shp/HydroRIVERS_v10_sa.shp')
geo1_path = Path('/home/jec18/datasets/gw/BGSmap/27R2R3_VUB.shp')
geo2_path = Path('/home/jec18/datasets/gw/BGSmap/VUB_GEO2.shp')


## Catchment + buffered bounding box
- Load km105 catchment, default CRS fallback to EPSG:4326 if missing.
- Reproject to PSAD, build bbox with 2 km buffer, and save to `reprojected/vectors`.

In [2]:
catch = gpd.read_file(catchment_path)
if catch.crs is None:
    catch = catch.set_crs('EPSG:4326')
catch_psad = catch.to_crs(PSAD_CRS)

bbox_geom = box(*catch_psad.total_bounds).buffer(BUFFER_METERS)
bbox_gdf = gpd.GeoDataFrame({"name": ["km105_bbox_buffered"]}, geometry=[bbox_geom], crs=PSAD_CRS)

catch_out = vectors_dir / 'km105_psad.gpkg'
bbox_out = vectors_dir / 'km105_bbox_buffered.gpkg'
catch_psad.to_file(catch_out, driver='GPKG')
bbox_gdf.to_file(bbox_out, driver='GPKG')

bbox_wgs84 = bbox_gdf.to_crs('EPSG:4326')
print('Catchment saved to', catch_out)
print('Buffered bbox saved to', bbox_out)
print('Buffered bbox (WGS84) bounds:', bbox_wgs84.total_bounds)


Catchment saved to /home/jec18/modflow/jupy/VUB/mountain/reprojected/vectors/km105_psad.gpkg
Buffered bbox saved to /home/jec18/modflow/jupy/VUB/mountain/reprojected/vectors/km105_bbox_buffered.gpkg
Buffered bbox (WGS84) bounds: [-72.59762339 -14.66623321 -70.76786373 -13.06896292]


## Elevation (HydroSheds DEM)
Clip South America DEM to the buffered bbox (in source CRS), reproject to PSAD, and save under `reprojected/rasters`.

In [3]:
dem = rxr.open_rasterio(dem_path, masked=True)
# Clip in the DEM CRS to avoid reprojecting the whole continent
bbox_in_dem_crs = bbox_gdf.to_crs(dem.rio.crs)
minx, miny, maxx, maxy = bbox_in_dem_crs.total_bounds
dem_clip = dem.rio.clip_box(minx=minx, miny=miny, maxx=maxx, maxy=maxy)

# Reproject the clipped window to PSAD
dem_psad = dem_clip.rio.reproject(PSAD_CRS)
dem_out = rasters_dir / 'dem_sa_clip_psad.tif'
dem_psad.rio.to_raster(dem_out)
print('DEM saved to', dem_out)
dem_psad


DEM saved to /home/jec18/modflow/jupy/VUB/mountain/reprojected/rasters/dem_sa_clip_psad.tif


<xarray.DataArray (band: 1, y: 1954, x: 2184)> Size: 17MB
array([[[ nan,  nan,  nan, ..., 474.,  nan,  nan],
        [ nan,  nan,  nan, ..., 491.,  nan,  nan],
        [ nan,  nan,  nan, ..., 525.,  nan,  nan],
        ...,
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan]]], dtype=float32)
Coordinates:
  * band         (band) int64 8B 1
  * y            (y) float64 16kB 6.459e+05 6.458e+05 ... 4.683e+05 4.682e+05
  * x            (x) float64 17kB 1.097e+06 1.097e+06 ... 1.295e+06 1.295e+06
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:       Area
    RepresentationType:  THEMATIC
    scale_factor:        1.0
    add_offset:          0.0
    long_name:           Band_1

## Rivers (HydroRIVERS)
Clip HydroRIVERS to the bbox and reproject to PSAD.

In [4]:
# Read only the bbox extent to keep memory reasonable
rivers_raw = gpd.read_file(rivers_path, bbox=tuple(bbox_in_dem_crs.total_bounds))
rivers_psad = rivers_raw.to_crs(PSAD_CRS)
rivers_clip = gpd.clip(rivers_psad, bbox_gdf)

rivers_out = vectors_dir / 'rivers_hydrorivers_psad.gpkg'
rivers_clip.to_file(rivers_out, driver='GPKG')
print('Clipped rivers saved to', rivers_out)
rivers_clip.head()


Clipped rivers saved to /home/jec18/modflow/jupy/VUB/mountain/reprojected/vectors/rivers_hydrorivers_psad.gpkg


,HYRIV_ID,NEXT_DOWN,MAIN_RIV,LENGTH_KM,DIST_DN_KM,DIST_UP_KM,CATCH_SKM,UPLAND_SKM,ENDORHEIC,DIS_AV_CMS,ORD_STRA,ORD_CLAS,ORD_FLOW,HYBAS_L12,geometry
3778,61180949,61181198,61280216,5.79,1055.6,8.8,24.51,24.5,1,0.457,1,5,7,6121201080,"LINESTRING (1260496.18 470937.752, 1261170.962..."
3689,61179511,61179044,60443230,2.63,5787.5,5.7,15.16,15.2,0,0.288,1,5,7,6120593140,"LINESTRING (1260708.538 475546.665, 1260706.74..."
3679,61179285,61179044,60443230,1.34,5787.5,4.7,12.67,12.7,0,0.236,1,6,7,6120593140,"LINESTRING (1262501.206 476472.917, 1262499.46..."
3711,61179956,61178173,60443230,10.62,5780.7,13.4,50.06,50.1,0,0.977,1,5,7,6120593140,"LINESTRING (1256001.159 473921.06, 1254879.353..."
3664,61179044,61178714,60443230,1.75,5785.5,7.6,4.36,32.2,0,0.607,2,5,7,6120593140,"LINESTRING (1262049.518 477623.859, 1261149.57..."


## Geology (BGS maps)
Load BGS layers, reproject to PSAD, clip to bbox, classify, and prepare rasters of classes and K values aligned to the DEM.

In [5]:
# Load and merge geology
geo1 = gpd.read_file(geo1_path)
geo2 = gpd.read_file(geo2_path)
if geo1.crs is None:
    raise ValueError('Missing CRS for geo1; set it before proceeding.')
if geo2.crs is None:
    raise ValueError('Missing CRS for geo2; set it before proceeding.')

# Reproject each source to a common CRS before merging
geo1_psad = geo1.to_crs(PSAD_CRS)
geo2_psad = geo2.to_crs(PSAD_CRS)
geo = pd.concat([geo1_psad, geo2_psad], ignore_index=True)

# Fix simple validity issues for rasterization
geo['geometry'] = geo.geometry.buffer(0)
geo_clip = gpd.clip(geo, bbox_gdf)

# Class mapping (from recharge_analysis)
reclass_map = {
    'Fractured Sedimentary': 3,
    'Glaciofluvial deposits': 2,
    'Fractured Volcanic': 3,
    'Metamorphic': 4,
    'Intrusive': 4,
    'Colluvial deposits': 1,
    'Alluvial deposits': 1,
    'Glacial deposits': 2,
    'Sedimentary': 4,
    'Non-consolidated porous': 1,
    'Fractured Volcano-Sedimentary': 3,
    'Volcanic': 4,
    'Lacustrine deposits': 1,
    'Fluvial deposits': 1,
    'Karstic': 5,
    'Lake': 1,
    'Sedimentary/Karstic': 5,
    'Volcano-Sedimentary': 4,
    'Biogenic deposits': 1,
    'Alluvial - fluvial deposits': 1,
    'Glacier': 2,
}

k_values = {
    1: 1e-5,  # unconsolidated
    2: 1e-4,  # glacial
    3: 1e-7,  # fractured
    4: 1e-8,  # aquitard
    5: 1e-5,  # karstic
}

geo_clip['HydroGeo_clean'] = geo_clip['HydroGeo'].str.strip()
geo_clip['hydro_class_numeric'] = geo_clip['HydroGeo_clean'].map(reclass_map)
unmapped = geo_clip[geo_clip['hydro_class_numeric'].isna()]['HydroGeo_clean'].dropna().unique()
if len(unmapped):
    print('Warning: unmapped HydroGeo classes ->', unmapped)
geo_clip['k_value'] = geo_clip['hydro_class_numeric'].map(k_values)

# Keep only safe, necessary columns for export to avoid field errors
cols_to_keep = [c for c in ['HydroGeo', 'HydroClass', 'HydroGeo_clean', 'hydro_class_numeric', 'k_value'] if c in geo_clip.columns]
geo_export = geo_clip[cols_to_keep + ['geometry']].copy()

geology_out = vectors_dir / 'geology_bgs_psad.gpkg'
geo_export.to_file(geology_out, driver='GPKG')
print('Clipped + classified geology saved to', geology_out)
geo_export[['HydroGeo_clean', 'hydro_class_numeric', 'k_value']].head()


Clipped + classified geology saved to /home/jec18/modflow/jupy/VUB/mountain/reprojected/vectors/geology_bgs_psad.gpkg


,HydroGeo_clean,hydro_class_numeric,k_value
476,Fractured Volcano-Sedimentary,3,1.000000e-07
511,Fluvial deposits,1,1.000000e-05
478,Sedimentary,4,1.000000e-08
495,Colluvial deposits,1,1.000000e-05
479,Fractured Volcano-Sedimentary,3,1.000000e-07


In [6]:
# Rasterize classes and K using the DEM grid as template
classes_path = rasters_dir / 'geology_classes_psad.tif'
k_path = rasters_dir / 'geology_k_psad.tif'

dem_src = rasterio.open(rasters_dir / 'dem_sa_clip_psad.tif')
profile = dem_src.profile.copy()
profile.update(count=1, compress='lzw')

shapes = [
    (geom, int(cls))
    for geom, cls in zip(geo_clip.geometry, geo_clip['hydro_class_numeric'])
    if not pd.isna(cls) and geom is not None and not geom.is_empty
]

class_profile = profile.copy()
class_profile.update(dtype='int32', nodata=0)
class_raster = features.rasterize(
    shapes=shapes,
    out_shape=(dem_src.height, dem_src.width),
    transform=dem_src.transform,
    fill=0,
    dtype='int32',
)
with rasterio.open(classes_path, 'w', **class_profile) as dst:
    dst.write(class_raster, 1)

k_profile = profile.copy()
k_profile.update(dtype='float32', nodata=-9999.0)
k_raster = np.full(class_raster.shape, -9999.0, dtype='float32')
for cls, kval in k_values.items():
    k_raster[class_raster == cls] = kval
with rasterio.open(k_path, 'w', **k_profile) as dst:
    dst.write(k_raster, 1)

print('Geology class raster ->', classes_path)
print('K raster ->', k_path)


Geology class raster -> /home/jec18/modflow/jupy/VUB/mountain/reprojected/rasters/geology_classes_psad.tif
K raster -> /home/jec18/modflow/jupy/VUB/mountain/reprojected/rasters/geology_k_psad.tif
